In [21]:
from keras.layers import Dense,Lambda,Flatten,InputLayer
from keras.models import Model
# from keras.applications.vgg19 import VGG19
# from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [22]:
IMAGE_SIZE=[128,128]
train_path="./Training/"
test_path="./Testing/"

# vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# for layer in vgg.layers:
#     layer.trainable=False

folders=glob("./Training/*")

# x=Flatten()(vgg.output)
# prediction=Dense(len(folders),activation="softmax")(x)

# model = Model(inputs=vgg.input, outputs=prediction)            

In [23]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(32, (2,2), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dropout(.4),
    tf.keras.layers.Dense(len(folders), activation='softmax')  
])

In [24]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [25]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255)
                                #    shear_range = 0.2,
                                #    zoom_range = 0.2,
                                #    horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('./Training/',
                                                 target_size = (128, 128),
                                                 batch_size = 50    ,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('./Testing/',
                                            target_size = (128, 128),
                                            batch_size = 50,
                                            class_mode = 'categorical')

Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [19]:
print(training_set.class_indices)

{'glioma_tumor': 0, 'meningioma_tumor': 1, 'no_tumor': 2, 'pituitary_tumor': 3}


In [26]:
r = model.fit(
training_set,
epochs=10,
validation_data=test_set,
steps_per_epoch=len(training_set),
validation_steps=len(test_set)
)

Epoch 1/10
58/58 [==============================] - 95s 2s/step - loss: 1.1876 - accuracy: 0.4694 - val_loss: 2.0067 - val_accuracy: 0.3680
Epoch 2/10
58/58 [==============================] - 87s 1s/step - loss: 0.6190 - accuracy: 0.7357 - val_loss: 2.1317 - val_accuracy: 0.5228
Epoch 3/10
58/58 [==============================] - 88s 2s/step - loss: 0.4284 - accuracy: 0.8291 - val_loss: 2.5059 - val_accuracy: 0.5533
Epoch 4/10
58/58 [==============================] - 84s 1s/step - loss: 0.3097 - accuracy: 0.8942 - val_loss: 2.6253 - val_accuracy: 0.5711
Epoch 5/10
58/58 [==============================] - 94s 2s/step - loss: 0.2029 - accuracy: 0.9185 - val_loss: 3.1524 - val_accuracy: 0.6904
Epoch 6/10
58/58 [==============================] - 93s 2s/step - loss: 0.1402 - accuracy: 0.9443 - val_loss: 3.1042 - val_accuracy: 0.7360
Epoch 7/10
58/58 [==============================] - 90s 2s/step - loss: 0.1045 - accuracy: 0.9594 - val_loss: 4.2398 - val_accuracy: 0.7132
Epoch 8/10
58/58 [==

In [27]:
model.save("brain_tumor_classifier.h5")